# Thread Detector

## Informa de métodos de aprendizaje supervisado

A continuación se evalúan varios métodos de aprendisaje supervisado con el propósito de encontrar el 
mejor indicado para realizar una deteción de **spam** o **no spam**

 Se importan las librerías necesarias

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import mglearn
import graphviz
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.neural_network import MLPClassifier

In [2]:
def indices_general(MC, names = None):
    precision_global = np.sum(MC.diagonal()) / np.sum(MC)
    error_global = 1 - precision_global
    precision_categoria  = pd.DataFrame(MC.diagonal()/np.sum(MC,axis = 1)).T
    if names!=None:
        precision_categoria.columns = names
    return {"Matriz de Confusion":MC, 
            "Precision Global":precision_global, 
            "Error Global":error_global, 
            "Precisión por categoria":precision_categoria}

Se caraga en un objecto carga el dataset correspondiente a **spambase.csv**

In [3]:
datos = pd.read_csv('spambase/spambase.csv',delimiter=',',decimal=".")
'Rows: %s Col: %s' % datos.shape

'Rows: 4601 Col: 58'

In [4]:
datos.head()

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,is_spam
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


Separate variables and predictions

In [5]:
X = datos.iloc[:,:54]
y = datos.iloc[:,57:58]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=0)

/home/aaron/Dropbox/Aaron/TEC/BASES_DE_DATOS_AVANZADAS/ThreatDetector/Thread_ML/venv/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2069: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


<h2> KNN </h2>

In [7]:
instancia_knn = KNeighborsClassifier(n_neighbors=3)

In [8]:
instancia_knn.fit(X_train,y_train)

/home/aaron/Dropbox/Aaron/TEC/BASES_DE_DATOS_AVANZADAS/ThreatDetector/Thread_ML/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  """Entry point for launching an IPython kernel.


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=3, p=2,
           weights='uniform')

In [9]:
print("Las predicciones en Testing son: {}".format(instancia_knn.predict(X_test)))

Las predicciones en Testing son: [1 0 0 ... 1 1 0]


In [10]:
print("Precisión en Testing: {:.2f}".format(instancia_knn.score(X_test, y_test)))

Precisión en Testing: 0.91


In [11]:
prediccion = instancia_knn.predict(X_test)
MC = confusion_matrix(y_test, prediccion)
print("Matriz de Confusión:\n{}".format(MC))


Matriz de Confusión:
[[764  58]
 [ 72 487]]


In [12]:
indices = indices_general(MC,list(np.unique(y)))
for k in indices:
    print("\n%s:\n%s"%(k,str(indices[k])))


Matriz de Confusion:
[[764  58]
 [ 72 487]]

Precision Global:
0.9058653149891384

Error Global:
0.09413468501086164

Precisión por categoria:
         0         1
0  0.92944  0.871199


<h2>  Árboles de Decisión </h2>

In [13]:
instancia_arbol = DecisionTreeClassifier(random_state=0)

In [14]:
instancia_arbol.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [15]:
print("Las predicciones en Testing son: {}".format(instancia_arbol.predict(X_test)))

Las predicciones en Testing son: [1 0 0 ... 0 0 0]


In [16]:
print("Precisión en Testing: {:.3f}".format(instancia_arbol.score(X_test, y_test)))

Precisión en Testing: 0.899


In [17]:
prediccion = instancia_arbol.predict(X_test)
MC = confusion_matrix(y_test, prediccion)
print("Matriz de Confusión:\n{}".format(MC))

Matriz de Confusión:
[[763  59]
 [ 81 478]]


In [18]:
indices = indices_general(MC,list(np.unique(y)))
for k in indices:
    print("\n%s:\n%s"%(k,str(indices[k])))


Matriz de Confusion:
[[763  59]
 [ 81 478]]

Precision Global:
0.8986241853729182

Error Global:
0.10137581462708178

Precisión por categoria:
          0         1
0  0.928224  0.855098


<h2> Bosques Aleatorios </h2>

In [19]:
instancia_bosque = RandomForestClassifier(n_estimators=10, random_state=0)


In [20]:
instancia_bosque.fit(X_train,y_train)


/home/aaron/Dropbox/Aaron/TEC/BASES_DE_DATOS_AVANZADAS/ThreatDetector/Thread_ML/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [21]:
print("Las predicciones en Testing son: {}".format(instancia_bosque.predict(X_test)))

Las predicciones en Testing son: [1 0 0 ... 0 1 0]


In [22]:
print("Precisión en Testing: {:.3f}".format(instancia_bosque.score(X_test, y_test)))

Precisión en Testing: 0.928


In [23]:
prediccion = instancia_bosque.predict(X_test)
MC = confusion_matrix(y_test, prediccion)
print("Matriz de Confusión:\n{}".format(MC))

Matriz de Confusión:
[[794  28]
 [ 72 487]]


In [24]:
indices = indices_general(MC,list(np.unique(y)))

In [25]:
for k in indices:
    print("\n%s:\n%s"%(k,str(indices[k])))


Matriz de Confusion:
[[794  28]
 [ 72 487]]

Precision Global:
0.9275887038377987

Error Global:
0.07241129616220132

Precisión por categoria:
          0         1
0  0.965937  0.871199


<h2>SVM</h2>

In [26]:
instancia_svm = svm.SVC(gamma='scale')

In [27]:
instancia_svm.fit(X_train,y_train)

/home/aaron/Dropbox/Aaron/TEC/BASES_DE_DATOS_AVANZADAS/ThreatDetector/Thread_ML/venv/lib/python3.6/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [28]:
print("Las predicciones en Testing son: {}".format(instancia_svm.predict(X_test)))

Las predicciones en Testing son: [1 0 0 ... 0 1 0]


In [29]:
print("Precisión en Testing: {:.3f}".format(instancia_svm.score(X_test, y_test)))

Precisión en Testing: 0.923


In [30]:
prediccion = instancia_svm.predict(X_test)
MC = confusion_matrix(y_test, prediccion)
print("Matriz de Confusión:\n{}".format(MC))

Matriz de Confusión:
[[786  36]
 [ 70 489]]


In [31]:
indices = indices_general(MC,list(np.unique(y)))
for k in indices:
    print("\n%s:\n%s"%(k,str(indices[k])))


Matriz de Confusion:
[[786  36]
 [ 70 489]]

Precision Global:
0.9232440260680667

Error Global:
0.07675597393193334

Precisión por categoria:
          0         1
0  0.956204  0.874776


<h2> Neural Networks </h2>

In [32]:
instancia_nl= MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(38, 185), random_state=1)

In [33]:
instancia_nl.fit(X_train,y_train)

/home/aaron/Dropbox/Aaron/TEC/BASES_DE_DATOS_AVANZADAS/ThreatDetector/Thread_ML/venv/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:916: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(38, 185), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [34]:
print("Las predicciones en Testing son: {}".format(instancia_nl.predict(X_test)))

Las predicciones en Testing son: [1 0 0 ... 0 1 0]


In [35]:
print("Precisión en Testing: {:.3f}".format(instancia_nl.score(X_test, y_test)))

Precisión en Testing: 0.917


In [36]:
prediccion = instancia_svm.predict(X_test)
MC = confusion_matrix(y_test, prediccion)
print("Matriz de Confusión:\n{}".format(MC))

Matriz de Confusión:
[[786  36]
 [ 70 489]]


In [37]:
indices = indices_general(MC,list(np.unique(y)))
for k in indices:
    print("\n%s:\n%s"%(k,str(indices[k])))


Matriz de Confusion:
[[786  36]
 [ 70 489]]

Precision Global:
0.9232440260680667

Error Global:
0.07675597393193334

Precisión por categoria:
          0         1
0  0.956204  0.874776


<h2> Instancia Bosque guardada</h2>

En el análisis realizado anteriormente es posiblle observar que el modelo con los mejores resulados corresponde a el de RandomForest. Este se va a guardar en disco con el propósito de que pueda ser utilizado posteriormente por otro servicio

In [38]:
from sklearn.externals import joblib
joblib.dump(instancia_bosque, 'random_forest.joblib') 

['random_forest.joblib']

In [39]:
clf = joblib.load('random_forest.joblib') 

In [53]:
clf.predict([[5]*54])

array([0])